In [13]:
import pandas as pd
import numpy as np
# import os
import jdc

In [20]:
class Mech():
    def __init__(self, chromosomes, sample_id):
        usecols = ["Chr", 
                   "Start", "End", "Ref", "Alt", "Func.refGene", "Otherinfo10", "Otherinfo11"]
        dtype = {"Chr":"category",
          "Start":np.int32,
          "End":np.int32,
          "Ref":"category",
          "Alt":"category",
          "Func.refGene":"category",
          "Otherinfo10":"category",
          "Otherinfo11":"string"}
        self.chromosomes = chromosomes
        self.sample_id = sample_id
        self.df = pd.concat(map(pd.read_csv, 
                    ["chr{}_{}.csv".format(chrom, self.sample_id) for chrom in self.chromosomes], 
                    dtype=dtype, usecols=usecols), ignore_index=True)
        col_names = self.df.columns.values.tolist()
        new_col_names = [''.join(char for char in name if char.isalnum()) for name in col_names]
        self.df.rename(columns=dict(zip(col_names, new_col_names)))

In [21]:
%%add_to Mech
def category_filter(self, **kwargs):
    for key, value in kwargs.items():
        cats = self.df[key].cat.categories
        self.df.remove_categories([x for x in cats if not x in value])
    self.df.dropna(inplace=True)

In [22]:
%%add_to Mech
def num_filter(self, **kwargs):
    pd.eval("DP = self.df.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=self.df, inplace=True)
    pd.eval("AF = self.df.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=self.df, inplace=True)
    for key, value in kwargs.items():
        self.df.loc[:,key].where(self.df.loc[:,key] > value, inplace=True)
    self.df.dropna(inplace=True)

In [ ]:
# idx_split([1,3,4,5,1,2,3,4]) = 4
def find_mid(indices):
    # creates a copy of indexes array but one index shifted to the left with first val -1
    # orig = [1,3,4,5,1,2,3,4]
    # copy = [0,1,3,4,5,1,2,3] 
    # sub = orig - copy = [1,2,1,1,-4,1,1,1]
    # np.argmin(sub) = 4
    copy = indices[:-1].copy()
    np.insert(copy, 0, indices[0] - 1)

In [3]:
def subtract(df1, df2):
    # functionalize outside of class
    uqid_df1 = df1[df1.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()
    uqid_df2 = df2[df2.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()  
    uqid = pd.concat([uqid_df1, uqid_df2], ignore_index=True)
    uqid_df1_len, uqid_df2_len = len(uqid_df1.index), len(uqid_df2.index)
    
    idx = np.append(np.arange(0, uqid_df1_len), np.arange(0, uqid_df2_len))
    uqid_df = pd.DataFrame({'idx':idx}, index=uqid)
    uqid_df.reset_index(inplace=True)
    uqid_df = uqid_df.sort_index().groupby('index').filter(lambda x: len(x) == 1)
    idx = uqid_df['idx'].to_numpy()
    # mid val functionalize outside of class
    mid_val = np.argmin(idx[len(idx)//5:]) + len(idx)//5
    # inv arr functionalize outside of class
    inv_arr = lambda max_val,idx_arr: np.array(sorted(set(range(0, max_val)).difference(idx_arr)))
    df1 = df1.drop(df1.index[inv_arr(uqid_df1_len, idx[:mid_val])])
    df2 = df2.drop(df2.index[inv_arr(uqid_df2_len, idx[mid_val:])])
    return df1

In [9]:
def init_df(chromomosomes_arg, sample_id, dtype_arg, usecols_arg):
    df = pd.concat(map(pd.read_csv, 
                       ["chr{}_{}.csv".format(chrom, sample_id) for chrom in chromosomes_arg], 
                      dtype=dtype+arg, usecols=usecols_arg), ignore_index=True)
    col_names = df.columns.values.tolist()
    # removes any symbols in col names
    new_col_names = [''.join(char for char in name if char.isalnum()) for name in col_names]
    df.rename(columns=dict(zip(col_names, new_col_names)))
    return df

In [ ]:
def main():
    ids = {'22241':['22334','22243']}
    chromosomes = ['1','2']
    for key, value in ids.items():
        parent = init_df(chromosomes, key, dtype, usecols)
        filt_parent = category_filter(parent, Otherinfo10='PASS')
        filt_parent = num_filter(filt_parent, DP=15, AF=.15)
        for val in value:
            variant = init_df(chromosomes, val, dtype, usecols)
            filt_variant = category_filter(variant, Otherinfo10='PASS')
            filt_variant = num_filter(filt_variant, DP=15, AF=.15)
            pv_result = subtract(filt_parent, variant)
            pv_result.to_csv("{}-{}.csv".format(key, val))
            vp_result = subtract(parent, filt_variant)
            vp_result.to_csv("{}-{}.csv".format(val, key))
    return None

In [ ]:
main()